<a href="https://colab.research.google.com/github/michelbarreto/contador_historias_infantis/blob/main/Contador_de_Estorias_Infantis_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instala dependências

In [1]:
!pip install -q -U google-generativeai
!pip install -q -U gTTS

Configura API do Gemini (lembre de inserir a secret API_KEY_GEMINI no Google Colab)

In [2]:
from google.colab import userdata
import google.generativeai as genai
import random
import datetime

API_KEY = userdata.get('API_KEY_GEMINI')
genai.configure(api_key=API_KEY)

In [3]:
safety_settings = {
    "HATE": "BLOCK_LOW_AND_ABOVE",
    "HARASSMENT": "BLOCK_LOW_AND_ABOVE",
    "SEXUAL": "BLOCK_LOW_AND_ABOVE",
    "DANGEROUS": "BLOCK_LOW_AND_ABOVE",
}

generation_config = {
    "temperature": 1,
    "candidate_count": 1
}

model = genai.GenerativeModel(model_name='gemini-1.0-pro', generation_config=generation_config, safety_settings=safety_settings)


Cria as funções auxiliares

In [4]:
def gerar_genero():
  generos = ["fantasia", "comédia infantil", "aventura infantil", "infantil", "conto de fadas"]

  return random.choice(generos)

In [5]:
# Função que retorna o tema do dia (para isso usa um array com 50 temas pré-definidos
# e retorna o índice que corresponde ao resto da divisão inteira entre "quantos dias passaram no ano" por 50)
def buscar_tema_do_dia():
  temas = [
      "A menina que conversava com os animais",
      "O gato que virou astronauta",
      "A viagem fantástica pela caixa de brinquedos",
      "O dragão que tinha medo de altura",
      "A princesa que não queria ser salva",
      "O mistério da floresta encantada",
      "A aventura no fundo do mar",
      "O dia em que os lápis de cor ganharam vida",
      "O ursinho de pelúcia que se perdia nas estrelas",
      "A canção mágica do passarinho",
      "O poder secreto das palavras",
      "A amizade entre o sol e a lua",
      "O menino que sonhava em voar",
      "A jornada em busca do arco-íris",
      "O segredo do castelo de chocolate",
      "A bailarina que dançava com as fadas",
      "O pirata que tinha medo da água",
      "A árvore mágica que contava histórias",
      "O mundo de cabeça para baixo",
      "O gigante gentil e o rato corajoso",
      "A cidade onde os sonhos se tornavam realidade",
      "O dia em que os brinquedos se rebelaram",
      "A super-heroína que usava óculos",
      "O monstro que só queria um abraço",
      "A corrida divertida dos caracóis",
      "O tesouro perdido no quintal",
      "A festa surpresa na floresta",
      "O presente especial da vovó",
      "O livro mágico que ensinava a voar",
      "O porquinho que queria ser bailarina",
      "O coelho que trocava de cor",
      "O reino das nuvens coloridas",
      "O menino que falava com o vento",
      "A menina que plantava estrelas",
      "O duende sapeca que morava no jardim",
      "O piquenique no planeta Marte",
      "A competição de canto dos passarinhos",
      "O reino da fantasia",
      "A abelha rainha e o favo de mel mágico",
      "O mistério do sumiço das meias",
      "A viagem no tempo com a máquina de lavar roupa",
      "O robô que queria ser humano",
      "A formiga trabalhadora e a cigarra cantora",
      "O pinguim que sonhava em voar",
      "A zebra que não tinha listras",
      "O elefante que tinha medo de ratos",
      "O leão que se apaixonou pela girafa"
  ]

  hoje = datetime.date.today()
  primeiro_dia_do_ano = datetime.date(hoje.year, 1, 1)
  dias_passados = (hoje - primeiro_dia_do_ano).days

  indice = dias_passados % len(temas)
  return temas[indice]

In [6]:
# Faz um pedido de sugestão de temas ao modelo Gemini
def pedir_sugestao_tema_ao_modelo():

  resp = model.generate_content(contents="Sugira um tema para uma história infantil. Seja criativo, sugira apenas o título, sem descrição.")
  return resp.text

In [7]:
# Faz o prompt para o modelo pedindo a criação de uma estória, conforme o tema e gênero
def pedir_estoria_ao_modelo(tema, genero):
  prompt = (f"Você é um criador de estórias infantis. Crie uma história sobre o tema {tema}. Utilize um gênero {genero}."
      "A história deve ser curta, possuir um final feliz e ter, no máximo, 5 parágrafos. Utilize palavras simples."
  )

  resp = model.generate_content(contents=prompt)
  return resp.text

In [8]:
# Transforma a estória (texto) em um áudio falado
def ler_estoria(estoria):
  if estoria == "":
    return

  from gtts import gTTS
  from IPython.display import Audio
  tts = gTTS(estoria, lang="pt", tld="com.br")
  tts.save('1.wav')
  sound_file = '1.wav'
  return Audio(sound_file, autoplay=True)

Gera a estoria

In [14]:
estoria = ""
genero = gerar_genero()

opcao_valida_escolhida = False

while not opcao_valida_escolhida:
  print((f"Bem-vindo! \n"
    "1) Ouvir a estória do dia \n"
    "2) Ouvir uma estória com tema sugerido pelo Gemini\n"
    "3) Digitar um tema e, após, ouvir uma estória sobre esse tema \n"
    "9) Sair"
  ))
  opcao = input("Escolha a opção abaixo: ")

  if(opcao not in ["1", "2", "3", "9"]):
    print("Opção inválida! \n\n")
    continue

  opcao_valida_escolhida = True

  tema = ""
  if(opcao == "1"):
    tema = buscar_tema_do_dia()

  elif(opcao == "2"):
    tema = pedir_sugestao_tema_ao_modelo()

  elif(opcao == "3"):
    tema = input("Informe um tema: ")

  elif(opcao == "9"):
    continue

  print(f"Gerando história de {tema} com o gênero {genero}.")
  estoria = pedir_estoria_ao_modelo(tema, genero)

  print(f"\n------\n {estoria} \n ---- \n")

ler_estoria(estoria)

Bem-vindo! 
1) Ouvir a estória do dia 
2) Ouvir uma estória com tema sugerido pelo Gemini
3) Digitar um tema e, após, ouvir uma estória sobre esse tema 
9) Sair
Escolha a opção abaixo: 2
Gerando história de A Fada das Borboletas Encantadas com o gênero comédia infantil.

------
 Era uma vez, numa floresta mágica, uma fada chamada Margarina. Margarina tinha um poder especial: ela fazia borboletas encantadas, que voavam e riam com uma alegria contagiante. Certo dia, enquanto Margarina brincava com suas borboletas, uma delas escapou e voou direto para um arbusto espinhoso.

Margarina tentou pegar a borboleta, mas ficou presa nos espinhos. As outras borboletas riram muito com a atrapalhação da fada. Elas então formaram uma nuvem colorida e voaram em volta de Margarina, fazendo cócegas nela e ajudando-a a se libertar.

A fada riu tanto que suas lágrimas de alegria brilharam como diamantes. As borboletas a abraçaram e cantaram uma música alegre, enchendo a floresta de felicidade. Margarina a